In [1]:
import pandas as pd
# pd.options.mode.chained_assignment = None  # default='warn'
import datetime
import random
import importlib
import os
import sys
import json
import time
import importlib
import math
import numpy as np
import pickle5 as pickle
import geopandas as gpd
import matplotlib.pyplot as plt
import osmnx as ox
from sklearn.cluster import KMeans
import networkx as nx

In [2]:
def cal_trust_score(file_path):
    rating_A = pd.read_csv(file_path)
#     print(rating_A)
    l_original = rating_A.drop(rating_A.columns[0], axis=1)
#     print(l_original)
    N  = len(l_original)
    Window = len(l_original.columns)
    temp = np.zeros(shape=(N,Window))
    x = pd.DataFrame(temp)
    cw = x.copy()
    w = x.copy()
    K = 4
    eeta = 2;
    M_BR = 4
    
    l1_original = l_original.apply(np.sort, axis = 1)
#     print(l1_original)
    for i in range(N):
        for j in range(Window):
            x.iloc[i,j] = 1 + ((K-1)*j)/Window
#     print(x)
    
    temp1 = np.zeros(shape=N)
    std_dr = pd.DataFrame(temp1)

    for i in range(N):
        std_dr.iloc[i] = np.std(l1_original[i])    

    for i in range(len(std_dr)):
        if int(std_dr.iloc[i]) == 0:
            std_dr.iloc[i] = np.mean(std_dr) 
#     print(std_dr)
    for i in range(N):
        for j in range(Window):
            cw.iloc[i,j] = (1/(math.sqrt(2*3.1415)*std_dr.iloc[i,0]))*(math.exp((-1*math.pow((x.iloc[i,j]-M_BR),2))/(2*math.pow(std_dr.iloc[i,0],2))))
#     print(cw)  

    for i in range(N):
        for j in range(Window):
            w.iloc[i,j] = cw.iloc[i,j]/np.sum(cw.iloc[i,:])
#     print(w)  
    
    R = np.zeros(shape=N)
    for segment in range(N):
        temp2 = np.zeros(shape=(4,Window))
        I = pd.DataFrame(temp2)
        #creates the evasion discrete levels.
        for j in range(Window):
            if l_original.iloc[segment,j] == 1:
                I.iloc[0,j] = 1
            elif l_original.iloc[segment,j] == 2:
                I.iloc[1,j] = 1
            elif l_original.iloc[segment,j] == 3:
                I.iloc[2,j] = 1
            else:
                I.iloc[3,j] = 1

        temp3 = np.zeros(shape=4)
        wd = pd.DataFrame(temp3)

        for i in range(4):
            for j in range(Window):
                wd.iloc[i,0] = wd.iloc[i,0] + I.iloc[i,j]*w.iloc[segment,j]

        for i in range(4):
            R[segment] = R[segment] + (i+1)*wd.iloc[i,0]
#     print(R)
    TR = np.zeros(shape=N)
    for i in range(N):
        TR[i] = (1/math.pow(K,eeta))*(math.pow(R[i],eeta))
#     print(TR)
    return TR,rating_A.iloc[:, 0].tolist()


In [3]:
list_of_ratings_per_cluster_per_incident_dir  =  "../data/rating/25_ratings_30min"
files_25_ratings = os.listdir(list_of_ratings_per_cluster_per_incident_dir)

In [4]:
trust_score=[]
prediction =[] #0 means high trusted 1 means less trusted 
segment_list_per_cluster = []
for file_path in files_25_ratings:
    fp = os.path.join(list_of_ratings_per_cluster_per_incident_dir, file_path)
    TR,list_segments = cal_trust_score(fp)
    trust_score.append(TR)
    segment_list_per_cluster.append(list_segments)
    kmeans = KMeans(n_clusters=2)
    kmeans.fit(TR.reshape(-1,1))
    y_kmeans = kmeans.predict(TR.reshape(-1,1))
    prediction.append(y_kmeans)
    print(y_kmeans)

[1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0
 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 1 0
 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1]
[0 0 1 1 1 1 0 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1
 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1]
[1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 1 0 1
 1 0 1 1 1 1]
[1 1 0 0 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0
 1 1 0]
[1 0 1 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1
 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1
 1 1 0 1 0 1 1 1 0]
[0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1
 0 0 0 0 0 0 0]
[0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 1 1 1 0 1 1
 0 1 1 0 0 0 1 0 0 0 

In [5]:
def get_neighboring_segments(G, cluster_list, segment, level=5, undirected=False):
    neighbor_df = {'XDSegID': [], 'level': []}
    done = []
    for l in range(level + 1):
        H = nx.ego_graph(L_XDSegID, n=segment, radius=l, undirected=undirected)
        for n in H.nodes:
            if n not in done:
                if n in cluster_list:
#                     print(n, l)
                    neighbor_df['XDSegID'].append(n)
                    neighbor_df['level'].append(l)
                    done.append(n)
    return pd.DataFrame(neighbor_df).sort_values(by='level')

In [6]:
if not os.path.exists(os.path.join(os.getcwd(), '../data')):
    raise OSError("Must first download data, see README.md")
data_dir = os.path.join(os.getcwd(), '../data')

if not os.path.exists(os.path.join(data_dir, 'segment_network_graphs')):
    os.mkdir(os.path.join(data_dir, 'segment_network_graphs'))
graphs_dir = os.path.join(data_dir, 'segment_network_graphs')
fp = os.path.join(graphs_dir, 'line_segment_graph.gml')
L_XDSegID = nx.read_gml(fp, destringizer=int)
print(nx.info(L_XDSegID))

MultiDiGraph with 6906 nodes and 15264 edges


In [7]:
ground_truth_data_path  =  "../data/generated_clusters/maxr085_incident_ratios/incidents_GT"
files_GT = os.listdir(ground_truth_data_path)
incident_GT = []
i = 0
while i< len(files_GT):
    fp = os.path.join(ground_truth_data_path, files_GT[i])
    with open(fp, 'rb') as handle:
        incident_GT.append( pickle.load(handle))
    i+=1
incident_GT_Frame = pd.concat(incident_GT)

In [8]:
fp_cluster = '../data/generated_clusters/optimized_clustering_0.7_0.85_maxr085_restricted.pkl'

# optimized_clustering_0.7_0.85_maxr085_restricted.pkl
# used_clusters_list_maxr085_25C_10-21-2021.pkl
with open(fp_cluster, 'rb') as handle:
    cluster_list = pickle.load(handle)

In [13]:
neighbour_segments_per_incident_per_cluster = []
list_of_cluster = []
for i in range(0,len(files_25_ratings)): 
#     160836160_20191130115000_20191130121500_ratings.csv
#     print(files_25_ratings[i])
    base = 0
    cluster_head = files_25_ratings[i][base:base+10]
    if(cluster_head.isnumeric() == True):
        month = files_25_ratings[i][base+15:base+17]
        day = files_25_ratings[i][base+17:base+19]
    else:
        cluster_head = files_25_ratings[i][base:base+9]
        month = files_25_ratings[i][base+14:base+16]
        day = files_25_ratings[i][base+16:base+18]
    list_of_cluster.append(cluster_head)

#     start_hour = files_25_ratings[i][19:21]
#     start_minute = files_25_ratings[i][21:23]
#     end_hour = files_25_ratings[i][34:36]
#     end_minute = files_25_ratings[i][36:38]
#     print(cluster_head,', ', month,' ,',day,',')#start_hour,',',start_minute,',',end_hour,',',end_minute)
#     print(cluster_head)
    incident_segment_frame = incident_GT_Frame[(incident_GT_Frame['cluster_head'] == int(cluster_head)) &\
                         ((incident_GT_Frame.index.month == int(month))&( incident_GT_Frame.index.day==int(day)))]
#     print(incident_segment_frame)
    if(len(incident_segment_frame) == 0): 
        continue
    segment_to_check = incident_segment_frame['XDSegID'].tolist()[0]
#     print(segment_to_check)
    level = 4
    neighbour_segments = get_neighboring_segments(L_XDSegID, cluster_list[int(cluster_head)], segment_to_check, level, undirected=True)
#     neighbour_segments2 = get_neighboring_segments(L_XDSegID, cluster_list[int(cluster_head)], segment_to_check, level, undirected=False)
#     neighbour_segments = pd.concat([neighbour_segments1,neighbour_segments1])
    neighbour_segments_per_incident_per_cluster.append(neighbour_segments)

In [14]:
len(neighbour_segments_per_incident_per_cluster)

25

In [15]:
success_rate_sum = 0
for fI in range(0,len(files_25_ratings)):
    print('*************',list_of_cluster[fI],'***************')
    TR_prediction = prediction[fI]
#     print(TR_prediction)
    neighbour_segments_frame =  neighbour_segments_per_incident_per_cluster[fI]
    neighbour_segments_list = neighbour_segments_frame['XDSegID'].tolist()
#     print(neighbour_segments_list)
    segment_list = segment_list_per_cluster[fI]
#     print(len(segment_list))
#     print(len(TR_prediction))
#     print( neighbour_segments_frame[neighbour_segments_frame['level']== 1])
    actual_affected_segment_count = neighbour_segments_frame[neighbour_segments_frame['level']== 1]['XDSegID'].count()
#     print("Actual affected segment count for ",fI,"-th Cluster is: ",actual_affected_segment_count)
    if(actual_affected_segment_count == 0):
        print(fI,"th segment has count zero",actual_affected_segment_count)
    predicted_affected_segment_count = 0
    for i in range(0,len(TR_prediction)):
        if(TR_prediction[i] == 1 ):
            if(segment_list[i]  in neighbour_segments_list ):
#                 print(neighbour_segments_frame[neighbour_segments_frame['XDSegID']==segment_list[i]]['level'].tolist())
                label = neighbour_segments_frame[neighbour_segments_frame['XDSegID']==segment_list[i]]['level'].tolist()[0]
                print(segment_list[i],':',label)
                if(label == int(1)):
                    predicted_affected_segment_count = predicted_affected_segment_count + 1
    success_rate = predicted_affected_segment_count/actual_affected_segment_count
    if(math.isnan(success_rate) == False):
        success_rate_sum = success_rate_sum + success_rate
        print(success_rate,predicted_affected_segment_count,actual_affected_segment_count)
#     break
# print(success_rate_sum)
# avg_success_rate = success_rate_sum/(len(files_25_ratings))
# print(avg_success_rate)

************* 1524274521 ***************
1524428760 : 4
1524639049 : 4
0.0 0 3
************* 1524373007 ***************
156550137 : 4
1524326614 : 3
1524329182 : 3
1524329256 : 1
1524333840 : 3
1524363122 : 4
1524369146 : 1
1524373007 : 1
1524373060 : 0
1524510950 : 4
1524544615 : 2
1524585576 : 1
1.0 4 4
************* 1524393544 ***************
429334038 : 3
449626432 : 4
1524393684 : 1
1524471115 : 2
0.3333333333333333 1 3
************* 1524394465 ***************
429359907 : 3
1524277270 : 4
1524308373 : 3
1524338317 : 2
1524379735 : 1
1524388271 : 3
1524424134 : 0
1524490899 : 1
1524507202 : 4
1524507307 : 1
1524545133 : 2
1524545200 : 4
1.0 3 3
************* 1524402419 ***************
1524402419 : 4
1524407671 : 3
1524407697 : 2
1524423918 : 4
1524466730 : 0
0.0 0 1
************* 1524423643 ***************
156189502 : 4
156428657 : 3
429334638 : 4
429334640 : 2
429334642 : 2
429334643 : 3
429334645 : 3
429334646 : 4
429334647 : 3
449626565 : 1
449626566 : 2
449626567 : 3
449626600 

C:\Users\jami1\anaconda3\envs\detection\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in longlong_scalars
